# Project Part 1

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/sgeinitz/cs39aa_project/blob/main/project_part1.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sgeinitz/cs39aa_project/blob/main/project_part1.ipynb)

## Introduction/Background

The dataset that I am using is the Coronavirus tweets NLP - Text Classification from kaggle. The data is from twitter and it is manually tagged with the tweet informations such as @ handles, date, tweet content and with name and @ handles replaced with code to avoid any privacy concerns. This is a text classification problem since
This is a __text classification__ problem since we are categorizing Sentiment random tweets into predetermined categories of sentiments: negative, positive and others. By definition, we are automatically analyzing and categorizing text into a number of predetermined categories.


## Exploratory Data Analysis

_You will now load the dataset and carry out some exploratory data analysis steps to better understand what text data looks like. See the examples from class on 10/. The following links provide some good resources of exploratory analyses of text data with Python._


* https://neptune.ai/blog/exploratory-data-analysis-natural-language-processing-tools
* https://regenerativetoday.com/exploratory-data-analysis-of-text-data-including-visualization-and-sentiment-analysis/
* https://medium.com/swlh/text-summarization-guide-exploratory-data-analysis-on-text-data-4e22ce2dd6ad  
* https://www.kdnuggets.com/2019/05/complete-exploratory-data-analysis-visualization-text-data.html  


In [ ]:
# import all of the python modules/packages you'll need here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
import seaborn as sns
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
import datasets
from datasets import Dataset , Sequence , Value , Features , ClassLabel , DatasetDict
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df_train = pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_train.csv",encoding='latin')
df_test = pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_test.csv',encoding='latin')

In [ ]:
print(df_train.shape)
print(df_train.columns)
df_train.head()

In [ ]:
df_train.drop(columns = ['UserName','ScreenName','Location','TweetAt'], inplace=True)
df_train.drop_duplicates(subset='OriginalTweet',inplace=True)

wordcloud = WordCloud(background_color='white', width=800, height=400).generate(''.join(df_train.v2))

plt.figure(figsize=(20, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
df_train["Sentiment"].value_counts()